In [ ]:
import json
import pandas as pd

# === STEP 1: Upload the JSON file in Colab ===
from google.colab import files
uploaded = files.upload()

with open(list(uploaded.keys())[0]) as f:
    pass_k_data = json.load(f)

# === STEP 2: Organize results per domain (C, H, I) ===
def extract_per_domain_tables(pass_k_data):
    domain_tables = {"C": [], "H": [], "I": []}

    for pass_k in ["pass_1", "pass_3", "pass_5"]:
        for model_key, metrics in pass_k_data.get(pass_k, {}).items():
            for suffix in ["_C", "_H", "_I"]:
                if suffix in model_key:
                    domain = suffix.strip("_")
                    model = model_key.replace(suffix, "").replace("(greedy)", "").strip()
                    break
            else:
                continue  # Skip if no domain suffix

            domain_tables[domain].append({
                "Model": model,
                "Pass@k": pass_k,
                "Class Success (%)": round(metrics.get("class_success", 0.0) * 100, 1),
                "Method Success (%)": round(metrics.get("fun_success", 0.0) * 100, 1)
            })

    return {
        domain: pd.DataFrame(data).sort_values(by=["Model", "Pass@k"])
        for domain, data in domain_tables.items()
    }

# === STEP 3: Show domain-wise tables ===
domain_tables = extract_per_domain_tables(pass_k_data)

for domain, df in domain_tables.items():
    print(f"\n📌 Domain: {domain}")
    display(df)
